In [1]:
import pandas as pd

# Alquiler

## Fase 1

Voy a dejar un poco más limpio el dataframe. Eliminaré variables que no aportan mucho (porque ya tienen un one-hot encoding, por ejemplo) y luego limpiaré aquellas observaciones que tienen `n_baths`=0.

In [2]:
dfm=pd.read_excel('data/pre-procesado/a-prov_mad-marzo24.xlsx')
dfb=pd.read_excel('data/pre-procesado/a-mun_bcn-marzo24.xlsx')
dfr=pd.read_excel('data/pre-procesado/a-rmb-sin_bcn-marzo24.xlsx')

Elimino `terrace_m2` porque no todos los pisos tienen información sobre su metraje. En cambio, sí sobre si tienen o no una terraza.

In [3]:
dfm=dfm.drop(['year_cons','floor','consum_EPC','emission_EPC','terrace_m2'],axis=1)
dfb=dfb.drop(['year_cons','floor','consum_EPC','emission_EPC','terrace_m2'],axis=1)
dfr=dfr.drop(['year_cons','floor','consum_EPC','emission_EPC','terrace_m2'],axis=1)

In [4]:
dfm = dfm[dfm['n_baths'] != 0]
dfb = dfb[dfb['n_baths'] != 0]
dfr = dfr[dfr['n_baths'] != 0]
print("Longitud de datos de Madrid:", len(dfm))
print("Longitud de datos de Barcelona municipio:", len(dfb))
print("Longitud de datos de la RMB sin Barcelona municipio:", len(dfr))

Longitud de datos de Madrid: 3645
Longitud de datos de Barcelona municipio: 3415
Longitud de datos de la RMB sin Barcelona municipio: 1646


Ahora elimino duplicados.

In [5]:
# Lista de columnas a mantener en cada DataFrame
columns_to_keep = ['codigo_inmueble', 'Lon_X', 'Lat_Y']

# Para dfm
columns_to_check_duplication_dfm = [col for col in dfm.columns if col not in columns_to_keep]
dfm.drop_duplicates(subset=columns_to_check_duplication_dfm, keep='first', inplace=True)

# Para dfb
columns_to_check_duplication_dfb = [col for col in dfb.columns if col not in columns_to_keep]
dfb.drop_duplicates(subset=columns_to_check_duplication_dfb, keep='first', inplace=True)

# Para dfr
columns_to_check_duplication_dfr = [col for col in dfr.columns if col not in columns_to_keep]
dfr.drop_duplicates(subset=columns_to_check_duplication_dfr, keep='first', inplace=True)

# Verificar la longitud de los DataFrames resultantes
print("Longitud de dfm después de eliminar duplicados:", len(dfm))
print("Longitud de dfb después de eliminar duplicados:", len(dfb))
print("Longitud de dfr después de eliminar duplicados:", len(dfr))

Longitud de dfm después de eliminar duplicados: 3645
Longitud de dfb después de eliminar duplicados: 3415
Longitud de dfr después de eliminar duplicados: 1595


## Multifamiliares

In [6]:
dfm = dfm[dfm['multifam'] == 1]
dfb = dfb[dfb['multifam'] == 1]
dfr = dfr[dfr['multifam'] == 1]
print("Longitud de dfm multifam:", len(dfm))
print("Longitud de dfb multifam:", len(dfb))
print("Longitud de dfr multifam:", len(dfr))

Longitud de dfm multifam: 2841
Longitud de dfb multifam: 2965
Longitud de dfr multifam: 976


## Etiquetas EPC

Reviso cuántos pisos tienen información de etiquetas EPC.

In [7]:
# Contar cuántas filas tienen valor 1 en la columna 'dummy_consum_EPC' de cada DataFrame
filas_con_1_consum_EPC_dfm = (dfm['dummy_consum_EPC'] == 1).sum()/len(dfm)*100
filas_con_1_consum_EPC_dfb = (dfb['dummy_consum_EPC'] == 1).sum()/len(dfb)*100
filas_con_1_consum_EPC_dfr = (dfr['dummy_consum_EPC'] == 1).sum()/len(dfr)*100

# Contar cuántas filas tienen valor 1 en la columna 'dummy_emission_EPC' de cada DataFrame
filas_con_1_emission_EPC_dfm = (dfm['dummy_emission_EPC'] == 1).sum()/len(dfm)*100
filas_con_1_emission_EPC_dfb = (dfb['dummy_emission_EPC'] == 1).sum()/len(dfb)*100
filas_con_1_emission_EPC_dfr = (dfr['dummy_emission_EPC'] == 1).sum()/len(dfr)*100

# Contar cuántas filas tienen valor 1 en ambas columnas 'dummy_consum_EPC' y 'dummy_emission_EPC' de cada DataFrame
filas_con_1_en_ambas_dfm = ((dfm['dummy_consum_EPC'] == 1) & (dfm['dummy_emission_EPC'] == 1)).sum()/len(dfm)*100
filas_con_1_en_ambas_dfb = ((dfb['dummy_consum_EPC'] == 1) & (dfb['dummy_emission_EPC'] == 1)).sum()/len(dfb)*100
filas_con_1_en_ambas_dfr = ((dfr['dummy_consum_EPC'] == 1) & (dfr['dummy_emission_EPC'] == 1)).sum()/len(dfr)*100

# Crear una tabla para mostrar el conteo por cada DataFrame
conteo_por_df = pd.DataFrame({
    'DataFrame': ['dfm', 'dfb', 'dfr'],
    'Total datos':[len(dfm),len(dfb),len(dfr)],
    'consum_EPC': [filas_con_1_consum_EPC_dfm, filas_con_1_consum_EPC_dfb, filas_con_1_consum_EPC_dfr],
    'emission_EPC': [filas_con_1_emission_EPC_dfm, filas_con_1_emission_EPC_dfb, filas_con_1_emission_EPC_dfr],
    'Ambas': [filas_con_1_en_ambas_dfm, filas_con_1_en_ambas_dfb, filas_con_1_en_ambas_dfr]
})

# Mostrar la tabla
print(conteo_por_df)

  DataFrame  Total datos  consum_EPC  emission_EPC      Ambas
0       dfm         2841   97.008096     97.008096  97.008096
1       dfb         2965   97.740304     97.740304  97.740304
2       dfr          976   98.360656     98.360656  98.360656


De los que tienen información, cuántos tienen valores inválidos (999, 0 o vacíos).

In [8]:
# Calcular el porcentaje de filas donde 'kwhm2_year' tiene valor 999, 0 o está vacía respecto a 'dummy_consum_EPC' con valor 1 para cada DataFrame
porcentaje_kwhm2_999_consum_dfm = ((dfm['dummy_consum_EPC'] == 1) & (dfm['kwhm2_year'].isin([999, 0, '']))).mean() * 100
porcentaje_kwhm2_999_consum_dfb = ((dfb['dummy_consum_EPC'] == 1) & (dfb['kwhm2_year'].isin([999, 0, '']))).mean() * 100
porcentaje_kwhm2_999_consum_dfr = ((dfr['dummy_consum_EPC'] == 1) & (dfr['kwhm2_year'].isin([999, 0, '']))).mean() * 100

# Calcular el porcentaje de filas donde 'kgCO2m2_year' tiene valor 999, 0 o está vacía respecto a 'dummy_emission_EPC' con valor 1 para cada DataFrame
porcentaje_kgCO2m2_999_emission_dfm = ((dfm['dummy_emission_EPC'] == 1) & (dfm['kgCO2m2_year'].isin([999, 0, '']))).mean() * 100
porcentaje_kgCO2m2_999_emission_dfb = ((dfb['dummy_emission_EPC'] == 1) & (dfb['kgCO2m2_year'].isin([999, 0, '']))).mean() * 100
porcentaje_kgCO2m2_999_emission_dfr = ((dfr['dummy_emission_EPC'] == 1) & (dfr['kgCO2m2_year'].isin([999, 0, '']))).mean() * 100

# Crear una tabla para mostrar los porcentajes por cada DataFrame
porcentaje_por_df = pd.DataFrame({
    'DataFrame': ['dfm', 'dfb', 'dfr'],
    "999 consum": [porcentaje_kwhm2_999_consum_dfm, porcentaje_kwhm2_999_consum_dfb, porcentaje_kwhm2_999_consum_dfr],
    "999 emission": [porcentaje_kgCO2m2_999_emission_dfm, porcentaje_kgCO2m2_999_emission_dfb, porcentaje_kgCO2m2_999_emission_dfr]
})

# Mostrar la tabla
print(porcentaje_por_df)

  DataFrame  999 consum  999 emission
0       dfm    2.604717           0.0
1       dfb    1.787521           0.0
2       dfr    0.819672           0.0


En general, es muy bajo el porcentaje. Vamos a eliminar esas observaciones.

In [9]:
# Eliminar filas con valor 0 en 'dummy_consum_EPC' y/o 'dummy_emission_EPC'
dfm = dfm[~((dfm['dummy_consum_EPC'] == 0) | (dfm['dummy_emission_EPC'] == 0))]
dfb = dfb[~((dfb['dummy_consum_EPC'] == 0) | (dfb['dummy_emission_EPC'] == 0))]
dfr = dfr[~((dfr['dummy_consum_EPC'] == 0) | (dfr['dummy_emission_EPC'] == 0))]

# Eliminar filas con valores 999, 0 o vacíos en 'kgCO2m2_year' y/o 'kwhm2_year'
dfm = dfm[~((dfm['kwhm2_year'].isin([999, 0, ''])) | (dfm['kgCO2m2_year'].isin([999, 0, ''])))]
dfb = dfb[~((dfb['kwhm2_year'].isin([999, 0, ''])) | (dfb['kgCO2m2_year'].isin([999, 0, ''])))]
dfr = dfr[~((dfr['kwhm2_year'].isin([999, 0, ''])) | (dfr['kgCO2m2_year'].isin([999, 0, ''])))]

# Mostrar la longitud de cada DataFrame después de eliminar las filas
print("Longitud de dfm después de eliminar filas:", len(dfm))
print("Longitud de dfb después de eliminar filas:", len(dfb))
print("Longitud de dfr después de eliminar filas:", len(dfr))

Longitud de dfm después de eliminar filas: 2682
Longitud de dfb después de eliminar filas: 2845
Longitud de dfr después de eliminar filas: 952


In [10]:
dfm.to_excel('data/a-prov_mad-marzo24.xlsx', index=False)
#Estos necesitarán un paso más, por eso los mantengo en pre-procesado
dfb.to_excel('data/pre-procesado/a-mun_bcn-marzo24-1.xlsx', index=False)
dfr.to_excel('data/pre-procesado/a-rmb-sin_bcn-marzo24-1.xlsx', index=False)